In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import pyvo as vo


## Checking column names for ref

In [4]:
from astroquery.gaia import Gaia
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')
print(gaiadr3_table)
for column in gaiadr3_table.columns:
  print(column.name)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152
solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pm

In [40]:
Gaia.ROW_LIMIT = -1  # Ensure the default row limit.
coord = SkyCoord(ra=0, dec=90, unit=(u.degree, u.degree), frame='icrs')
j = Gaia.cone_search_async(coord, radius=u.Quantity(7.0, u.deg), columns=("source_id", "ra", "dec", "phot_g_mean_flux", "phot_g_mean_flux_error", "pm", "parallax", "parallax_error", "phot_bp_mean_flux", "phot_bp_mean_flux_error", "phot_rp_mean_flux", "phot_rp_mean_flux_error", "teff_gspphot", "teff_gspphot_lower", "teff_gspphot_upper", "logg_gspphot", "logg_gspphot_lower", "logg_gspphot_upper", "mh_gspphot", "mh_gspphot_upper", "mh_gspphot_lower", "bp_rp", "bp_g", "g_rp",    ))  
r = j.get_results()
r.pprint()  
r = r.to_pandas()

INFO: Query finished. [astroquery.utils.tap.core]
     source_id              ra         ...    g_rp            dist        
                           deg         ...    mag                         
------------------- ------------------ ... ---------- --------------------
1729382222550672128 241.92634152467497 ...  1.0064201 0.009951722155818802
 576460717944283264  78.08426629765074 ...  1.2406654 0.010951303183265992
1729382226847271424  219.4936441916612 ...  0.9854889 0.013665525890330099
 576460687879588608 21.672688446110335 ...  1.1661091 0.016002142554133326
2305842974854213120  316.7325738867149 ...  0.8146248 0.017676970890539843
 576460649225998848  74.25170092702014 ...  1.1186504 0.020118063431011763
 576460653519850752  66.90480624712295 ... 0.99625015 0.021242262336965603
 576460687879588736  6.343432887888492 ...  1.0765495 0.021294111495708107
 576460687881492608 10.230702851198853 ...  0.7433739  0.02289992450889343
                ...                ... ...        

In [41]:
coord = SkyCoord(ra=0, dec=-90, unit=(u.degree, u.degree), frame='icrs')
j = Gaia.cone_search_async(coord, radius=u.Quantity(7.0, u.deg), columns=("source_id", "ra", "dec", "phot_g_mean_flux", "phot_g_mean_flux_error", "pm", "parallax", "parallax_error", "phot_bp_mean_flux", "phot_bp_mean_flux_error", "phot_rp_mean_flux", "phot_rp_mean_flux_error", "teff_gspphot", "teff_gspphot_lower", "teff_gspphot_upper", "logg_gspphot", "logg_gspphot_lower", "logg_gspphot_upper", "mh_gspphot", "mh_gspphot_upper", "mh_gspphot_lower", "bp_rp", "bp_g", "g_rp",    ))  
r2 = j.get_results()
r2.pprint()  
r2= r2.to_pandas()

INFO: Query finished. [astroquery.utils.tap.core]
     source_id              ra         ...    g_rp            dist        
                           deg         ...    mag                         
------------------- ------------------ ... ---------- --------------------
5188146770731873152 106.80984765520387 ...  1.1645947 0.007127426742523844
4611686018427432192  63.43077197057562 ...  0.9993496 0.011116634915285657
5764607527330473600 200.29128759031627 ...  1.1169605 0.011148145640936203
5764607527332179584  193.6332911023808 ...  0.5905762 0.011251922624535499
5188146770731865088 152.06766340953186 ...  0.7172222 0.011608567308333859
5188146775027112576 127.97748728182675 ... 0.62862396 0.012364108236569075
5764607527330473856 218.48376334824772 ...  0.9836693 0.013617281132369983
6341068275337710080  341.8515625323109 ... 0.82063484 0.013659476077543966
6341068313993334272 350.81582171409343 ...  1.0209637 0.013990962101832215
                ...                ... ...        

In [11]:
r = r.drop('pseudocolour', axis=1)
r = r.drop('pseudocolour_error', axis=1)

# Remove rows with any NULL values
result = r.dropna()

## Getting Simbad otypes for these bad boys

In [16]:
# Initialize SIMBAD
simbad = Simbad()
simbad.ROW_LIMIT = -1
simbad.add_votable_fields( "plx_value", "V", "I", "J", "H", "K","G", "pmdec", "pmra", "hpx","ids", "otype")

# Define coordinates (random)
coordinates = SkyCoord(0, -90, unit=("deg", "deg"))
coordinates2 = SkyCoord(0, 90, unit=("deg", "deg"))

# Query region 1st half
result = simbad.query_region(coordinates, radius="90d0m",
                             criteria="otype = 'Ma*..' OR otype = 'MS*..' OR otype = 'Y*O..' OR otype = 'Ev*..' ")

filtered_result = result["main_id", "ra", "dec", "plx_value", "V", "I", "J", "H", "K","G", "pmdec", "pmra", 
                         "hpx", "ids", "otype"]
result_df = filtered_result.to_pandas() # Convert result to a Pandas DataFrame
filtered_result = result_df.dropna() # Remove rows with any NULL values
data1 = filtered_result.to_numpy() # Convert to numpy array


# Query region 2nd half
result2 = simbad.query_region(coordinates2, radius="90d0m",
                             criteria="otype = 'Ma*..' OR otype = 'MS*..' OR otype = 'Y*O..' OR otype = 'Ev*..' ")

filtered_result2 = result2["main_id", "ra", "dec", "plx_value", "V", "I", "J", "H", "K","G", "pmdec", "pmra",
                            "hpx", "ids", "otype"]
result_df2 = filtered_result2.to_pandas() # Convert result to a Pandas DataFrame
filtered_result2 = result_df2.dropna() # Remove rows with any NULL values
data2 = filtered_result2.to_numpy() # Convert to numpy array

# Combine the two arrays & save
simbad_data =np.row_stack((data1, data2))
#np.save("fullsky4cats", data)

## Combining both

In [46]:
simbad_data = pd.concat((filtered_result, filtered_result2))
gaia_data = pd.concat((r, r2))
print(simbad_data.shape)
print(gaia_data.shape)

# Convert Gaia source_id to string
gaia_data['source_id'] = gaia_data['source_id'].astype(str)

# Filter SIMBAD data to only include rows where 'ids' contains 'Gaia DR3'
simbad_data['gaia_id'] = simbad_data['ids'].apply(lambda x: next((id for id in x.split('|') if id.startswith('Gaia DR3')), None))

simbad_data['gaia_id'] = simbad_data['gaia_id'].str.lstrip('Gaia DR3')
simbad_data = simbad_data.dropna(subset=['gaia_id'])

# Convert Gaia ID to integer
simbad_data['gaia_id'] = simbad_data['gaia_id'].astype(str)


print(simbad_data['gaia_id'])
# Merge Gaia and SIMBAD data on matching IDs
merged_data = pd.merge(r, simbad_data, left_on='source_id', right_on='gaia_id', how='inner')

# Display the merged data
print(merged_data)

(92568, 15)
(1854479, 25)
20        6018034958869558912
29         216493246970258176
52        4040046766417141504
53        4040054291143186944
65        4040144313740928768
                 ...         
401867     865401039307016448
401869     865401378607077888
401870     865401382903998976
401872     865401623422138368
401875     865401795220844160
Name: gaia_id, Length: 92320, dtype: object


ValueError: You are trying to merge on int64 and object columns for key 'source_id'. If you wish to proceed you should use pd.concat

In [49]:
def GetGAIAData(GaiaDR3SourceIDs):
    # gets the GAIA data for the provided GaiaDR3SourceIDs's
    # and writes to a local CSV
        
    dfGaia = pd.DataFrame()
    
    #job = Gaia.launch_job_async( "select top 100 * from gaiadr2.gaia_source where parallax>0 and parallax_over_error>3. ") # Select `good' parallaxes
    qry = "SELECT * FROM gaiadr3.gaia_source gs WHERE gs.source_id in (" + GaiaDR3SourceIDs + ");"
    
    job = Gaia.launch_job_async( qry )
    tblGaia = job.get_results()       #Astropy table
    dfGaia = tblGaia.to_pandas()      #convert to Pandas dataframe
    print(dfGaia)
    
    #npGAIARecords = dfGaia.to_numpy() #convert to numpy array    
    #lstGAIARecords = [list(x) for x in npGAIARecords]   #convert to List[]
    
    #FileForLocalStorage = FolderForLocalStorage + str(lstGAIARecords[0][2]) + '.csv'  # use SourceID from 1st record
    #dfGaia.to_csv (FileForLocalStorage, index = False, header=True)    
GetGAIAData(simbad_data['gaia_id'])

500 Error 500:
null


HTTPError: Error 500:
null

In [55]:
from astroquery.gaia import Gaia
import pandas as pd

def GetGAIAData(GaiaDR3SourceIDs):
    try:
        dfGaia = pd.DataFrame()
        qry = f"SELECT * FROM gaiadr3.gaia_source gs WHERE gs.source_id in ({GaiaDR3SourceIDs});"
        job = Gaia.launch_job_async(qry)
        tblGaia = job.get_results()
        dfGaia = tblGaia.to_pandas()
        print(dfGaia)
    except Exception as e:
        print(f"An error occurred: {e}")
simbadgaiaid = simbad_data['gaia_id'].str.cat(sep=', ')
GetGAIAData(simbad_data['gaia_id'].str.cat(sep=', '))

An error occurred: 500


In [58]:
def split_ids_into_chunks(id_string, chunk_size=1000):
    # Split the string into a list of IDs
    id_list = id_string.split(', ')
    
    # Create chunks of the specified size
    chunks = [', '.join(id_list[i:i + chunk_size]) for i in range(0, len(id_list), chunk_size)]
    
    return chunks

# Example usage
GaiaDR3SourceIDs = ', '.join(simbad_data['gaia_id'].astype(str))
chunks = split_ids_into_chunks(GaiaDR3SourceIDs)

# Print the chunks to verify
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}")

Chunk 1: 6018034958869558912, 216493246970258176, 4040046766417141504, 4040054291143186944, 4040144313740928768, 4040146169166551808, 4658829954619222272, 4040223199311502592, 4040225127817976960, 4040238906076712960, 4663636023007193984, 5983682367322707968, 5392345257042224512, 4182449048454297600, 4182449671226792448, 4655285747641775744, 4655080924926291072, 4657915882546574080, 4655035535726226560, 4657236281232142080, 4658077407683722880, 4651832456469009152, 4657078875032208128, 4657215184387101568, 4657034452147477888, 4657082109111827584, 4657218070604932992, 4657177732226892032, 4657229203122133760, 4657499545530867840, 4655394908490228608, 4655492146550876928, 4655521489771306112, 4655134702206049152, 4655551485825438848, 4655594366782409472, 4655425694817281024, 4655542621012120192, 4661574851040855040, 4043200234495068288, 4043245791132666240, 4661666179215129728, 4660111297993334656, 4662093923606908800, 6244083039015457152, 5607366983624065408, 4654995545239669120, 46615

In [60]:
def GetGAIAData(GaiaDR3SourceIDs):
    try:
        dfGaia = pd.DataFrame()
        qry = f"SELECT * FROM gaiadr3.gaia_source gs WHERE gs.source_id in ({GaiaDR3SourceIDs});"
        job = Gaia.launch_job_async(qry)
        tblGaia = job.get_results()
        dfGaia = tblGaia.to_pandas()
        print(dfGaia)
    except Exception as e:
        print(f"An error occurred: {e}")

def split_ids_into_chunks(id_string, chunk_size=10000):
    id_list = id_string.split(', ')
    chunks = [', '.join(id_list[i:i + chunk_size]) for i in range(0, len(id_list), chunk_size)]
    return chunks

# Example usage
GaiaDR3SourceIDs = ', '.join(simbad_data['gaia_id'].astype(str))
chunks = split_ids_into_chunks(GaiaDR3SourceIDs)

# Process each chunk
for chunk in chunks:
    GetGAIAData(chunk)

INFO: Query finished. [astroquery.utils.tap.core]
              solution_id                   designation            source_id  \
0     1636148068921376768  Gaia DR3 2340699799852817152  2340699799852817152   
1     1636148068921376768  Gaia DR3 2376629159789392256  2376629159789392256   
2     1636148068921376768  Gaia DR3 2379500362605986944  2379500362605986944   
3     1636148068921376768  Gaia DR3 2423264670603644800  2423264670603644800   
4     1636148068921376768  Gaia DR3 2893195936018751360  2893195936018751360   
...                   ...                           ...                  ...   
9488  1636148068921376768  Gaia DR3 6848431731223859456  6848431731223859456   
9489  1636148068921376768  Gaia DR3 6864617817991978624  6864617817991978624   
9490  1636148068921376768  Gaia DR3 6874536546746663552  6874536546746663552   
9491  1636148068921376768  Gaia DR3 6880067571269293056  6880067571269293056   
9492  1636148068921376768  Gaia DR3 6905479346572742016  6905479346572

# Getting gaia data for simbad sources 

In [62]:
def GetGAIAData(GaiaDR3SourceIDs):
    try:
        qry = f"SELECT * FROM gaiadr3.gaia_source gs WHERE gs.source_id in ({GaiaDR3SourceIDs});"
        job = Gaia.launch_job_async(qry)
        tblGaia = job.get_results()
        dfGaia = tblGaia.to_pandas()
        return dfGaia
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

def split_ids_into_chunks(id_string, chunk_size=10000):
    id_list = id_string.split(', ')
    chunks = [', '.join(id_list[i:i + chunk_size]) for i in range(0, len(id_list), chunk_size)]
    return chunks

# Example usage
GaiaDR3SourceIDs = ', '.join(simbad_data['gaia_id'].astype(str))
chunks = split_ids_into_chunks(GaiaDR3SourceIDs)

# Initialize an empty DataFrame to store all data
combined_df = pd.DataFrame()

# Process each chunk and append the results to the combined DataFrame
for chunk in chunks:
    dfGaia = GetGAIAData(chunk)
    combined_df = pd.concat([combined_df, dfGaia], ignore_index=True)

# Convert the combined DataFrame to a NumPy array if needed
combined_matrix = combined_df.to_numpy()

# Print the combined DataFrame and matrix to verify
print(combined_df)
print(combined_matrix)

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
               solution_id                   designation            source_id  \
0      1636148068921376768  Gaia DR3 2340699799852817152  2340699799852817152   
1      1636148068921376768  Gaia DR3 2376629159789392256  2376629159789392256   
2      1636148068921376768  Gaia DR3 2379500362605986944  2379500362605986944   
3      1636148068921376768  Gaia DR3 2423264670603644800  2423264670603644800   
4      1636148068921376768  Gaia DR3 2893195936018751360  2893195936018751360   
...           

In [69]:
useful_gaia = combined_df[["source_id", "ra", "dec", "phot_g_mean_flux", "phot_g_mean_flux_error", "pm", "parallax", "parallax_error", "phot_bp_mean_flux", "phot_bp_mean_flux_error", "phot_rp_mean_flux", "phot_rp_mean_flux_error", "teff_gspphot", "teff_gspphot_lower", "teff_gspphot_upper", "logg_gspphot", "logg_gspphot_lower", "logg_gspphot_upper", "mh_gspphot", "mh_gspphot_upper", "mh_gspphot_lower", "bp_rp", "bp_g", "g_rp", "ruwe", "duplicated_source"]]
useful_gaia2 = useful_gaia.dropna()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Combining both df

In [71]:
import pandas as pd

# Assuming filtered_result, filtered_result2, r, and r2 are already defined DataFrames

# Concatenate the DataFrames
simbad_data = pd.concat((filtered_result, filtered_result2))
gaia_data = useful_gaia2

# Convert Gaia source_id to string
gaia_data['source_id'] = gaia_data['source_id'].astype(str)

# Filter SIMBAD data to only include rows where 'ids' contains 'Gaia DR3'
simbad_data['gaia_id'] = simbad_data['ids'].apply(lambda x: next((id for id in x.split('|') if id.startswith('Gaia DR3')), None))

# Remove 'Gaia DR3' prefix and drop rows with NaN values in 'gaia_id'
simbad_data['gaia_id'] = simbad_data['gaia_id'].str.lstrip('Gaia DR3')
simbad_data = simbad_data.dropna(subset=['gaia_id'])

# Ensure 'gaia_id' is a string
simbad_data['gaia_id'] = simbad_data['gaia_id'].astype(str)

# Merge Gaia and SIMBAD data on matching IDs
merged_data = pd.merge(gaia_data, simbad_data, left_on='source_id', right_on='gaia_id', how='inner')

np.save("fullsky4catsgaia", merged_data)
# Display the merged data
print(merged_data)


C:\Users\jcwin\AppData\Local\Temp\ipykernel_26232\3698886038.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gaia_data['source_id'] = gaia_data['source_id'].astype(str)


                 source_id        ra_x      dec_x  phot_g_mean_flux  \
0      2340699799852817152    0.862927 -21.360054      1.086534e+05   
1      2379500362605986944  347.923830 -26.496008      1.727714e+06   
2      2893195936018751360   96.592078 -31.431350      3.280597e+05   
3      2948355655548214400  106.672032 -14.501737      1.754187e+05   
4      4036461911863364992  268.567064 -39.245435      3.871391e+04   
...                    ...         ...        ...               ...   
48461  4512766260168685696  287.354818  15.303529      4.745380e+04   
48462  4514789602058909312  286.505229  18.613976      2.126884e+04   
48463  4517243059169860480  284.126438  17.965214      1.683816e+04   
48464  4519217850760871808  283.323129  21.225805      2.643338e+06   
48465  4595217743723364864  262.279130  27.822616      1.051243e+05   

       phot_g_mean_flux_error         pm  parallax  parallax_error  \
0                  371.537720   2.190466  0.032278        0.017255   
1      

## Cleaning data

In [79]:
merged_data2 = np.delete(merged_data, 41, 1)
merged_data2 = np.delete(merged_data2, 39, 1)
merged_data2 = np.delete(merged_data2, 26, 1)
merged_data2 = np.delete(merged_data2, 25, 1)
np.save("fullsky4catsgaiac", merged_data2)

## Other trashy stuff